# Criando o dataset 'online_retail':

O banco de dados foi extraído em formato xlsx do [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/352/online+retail).

Com o pandas, criei um database em POSTGRESQL a partir do arquivo 'Online Retail.xlsx'.

In [5]:
%run ../src/create_dataset.py

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


# Limpando o dataset 'online_retail':

Em uma análise preliminar do dataset, foi observado valores NULL e outros erros que poderiam prejudicar a análise dos dados. Para isso, usei POSTGRESQL para limpar o banco de dados usando a seguinte querry:

```postgresql
CREATE TABLE IF NOT EXISTS online_retail_cleaned AS

SELECT
    -- Invoice: apenas os que começam com dígito
    invoice_no,

    -- StockCode: remove letras e mantém apenas códigos numéricos
    regexp_replace(stock_code, '[[:alpha:]]', '', 'g') AS stock_code_cleaned,

    -- Description: lowercase, trim e normaliza espaços
    lower(trim(regexp_replace(description, '[\s,]+', ' ', 'g'))) AS description_cleaned,

    -- Quantity: valor absoluto
    ABS(quantity) AS quantity_cleaned,

    -- InvoiceDate: sem alteração necessária
    invoice_date,

    -- UnitPrice: apenas valores positivos
    ABS(unit_price) AS unit_price_cleaned,

    -- CustomerID: mantido (filtrado no WHERE)
    customer_id,

    -- Country: lowercase + trim
    trim(lower(country)) AS country_cleaned

FROM online_retail
WHERE
    -- invoice começa com dígito
    invoice_no ~ '^[[:digit:]]'

    -- customer_id válido
    AND customer_id IS NOT NULL

    -- unit_price positivo
    AND unit_price > 0

    -- stock_code limpo com exatamente 5 dígitos
    AND length(regexp_replace(stock_code, '[[:alpha:]]', '', 'g')) = 5

    -- retirar pedidos cancelados (stock_code inicia com 'c')
    AND lower(stock_code) NOT LIKE 'c%';

# Criando o dataset 'online_retail_cleaned':

Com o pandas, criei um database em POSTGRESQL com as modificações anteriores a partir do arquivo 'Online Retail.xlsx'.

In [6]:
%run ../src/create_cleaned_dataset.py

,invoice_no,stock_code_cleaned,description_cleaned,quantity_cleaned,invoice_date,unit_price_cleaned,customer_id,country_cleaned
0,536783,84692,box of 24 cocktail parasols,50,2010-12-02 15:19:00,0.32,15061.0,united kingdom
1,536783,84347,rotating silver angels t-light hldr,120,2010-12-02 15:19:00,1.88,15061.0,united kingdom
2,536783,71459,hanging jam jar t-light holder,144,2010-12-02 15:19:00,0.72,15061.0,united kingdom
3,536783,48194,doormat hearts,70,2010-12-02 15:19:00,6.75,15061.0,united kingdom
4,536783,48184,doormat english rose,30,2010-12-02 15:19:00,6.75,15061.0,united kingdom
5,536783,35001,hand open shape gold,42,2010-12-02 15:19:00,3.39,15061.0,united kingdom
6,536783,22752,set 7 babushka nesting boxes,96,2010-12-02 15:19:00,7.65,15061.0,united kingdom
7,536783,22659,lunch box i love london,64,2010-12-02 15:19:00,1.65,15061.0,united kingdom
8,536783,22630,dolly girl lunch box,64,2010-12-02 15:19:00,1.65,15061.0,united kingdom
9,536783,22629,spaceboy lunch box,64,2010-12-02 15:19:00,1.65,15061.0,united kingdom
